Install requirements

In [1]:
!pip install deeplake
!pip install deeplake[audio]
!pip install PyAV

zsh:1: command not found: pip
zsh:1: no matches found: deeplake[audio]
zsh:1: command not found: pip
zsh:1: command not found: pip


Import libraries

In [ ]:
import deeplake
import librosa

Load the dataset and print the summary

In [2]:
ds = deeplake.load("hub://activeloop/nsynth-train")
ds.summary()

NameError: name 'deeplake' is not defined

Get audio details

In [ ]:
audio_tensor = ds['audios']

print("tensor:        ", "audios")
print("htype:         ", audio_tensor.htype)
print("shape:         ", audio_tensor.shape)
print("dtype:         ", audio_tensor.dtype)

tensor:         audios
htype:          audio
shape:          (289205, 64000, 1)
dtype:          float64


Listen to the audio

In [ ]:
import numpy as np
from IPython.display import Audio

# Extract the nth audio sample
n = 0  # This will be the first audio sample
audio_sample = ds['audios'][n].numpy().squeeze()  # Convert to numpy array and remove unnecessary dimension

# 16kHz rate
sample_rate = 16000

# Use IPython's Audio utility to play the audio
Audio(audio_sample, rate=sample_rate)

Only grab the dataset where instument == keyboard

In [19]:
filtered_ds_audio = []
filtered_ds_pitch = []
pitch_values = ds['pitch'].numpy().squeeze()

#for i, sample in enumerate(ds[:200]):
for i in range(len(ds)):
  instrument_family = ds[i].instrument_family.data()['text'][0]
  if instrument_family == 'keyboard':
    audio_sample = ds['audios'][i].numpy().squeeze()  # Convert to numpy array and remove unnecessary dimension

    # Trim the audio
    audio_sample = audio_sample[:int(3 * 16000)]
    # Convert audio to CQT (Constant-Q Transform)
    audio_sample = librosa.cqt(audio_sample, sr=16000)

    filtered_ds_audio.append(audio_sample)
    # Add modified pitch value to the list
    filtered_ds_pitch.append(pitch_values[i]-21)

#print("audio array: ", filtered_ds_audio, " \n")
print(len(filtered_ds_audio))
#print("pitch: ",filtered_ds_pitch, " \n")
print(len(filtered_ds_pitch))

KeyboardInterrupt: ignored

In [ ]:
# =================== Augmentation Function Library ==================
def noise_injection(audio, noise_factor = 0.001):
  '''Inject random/white noise into the audio'''
  noise = np.random.randn(len(audio))
  return audio + noise_factor * noise

def change_speed(audio, speed_factor = 2):
  '''
  Changes the speed of playback by scaling time
  speed_factor > 1      => speeds up playback
  0 < speed_factor < 1  => slows down playback
  '''
  return librosa.effects.time_stretch(audio, rate=speed_factor)

def change_speed_and_fit(audio, stretch_factor = 0.5):
  '''Changes speed similar to change_speed and fits into the same length as the input audio'''
  input_length = len(audio)
  stretched_audio = librosa.effects.time_stretch(audio.astype('float'), rate=stretch_factor)
  if len(stretched_audio) > input_length:
    return stretched_audio[:input_length]
  else:
    return np.pad(stretched_audio, (0, max(0, input_length - len(stretched_audio))), "constant")

def shift_pitch(audio, steps = 10, sr=16000):
  '''Shifts the pitch up by the specified amount of steps'''
  return librosa.effects.pitch_shift(audio, sr=sr, n_steps=steps)

def shift_time(audio, shift_factor = 0.5, random = False):
  '''Shifts the audio in time by a given factor, keeping the same length of the intial audio'''
  if random:
    shift_factor = shift_factor * 2 * (np.random.uniform() - 0.5)

  start = int(len(audio) * shift_factor)
  if (start > 0):
      return np.pad(audio,(start,0),mode='constant')[0:len(audio)]
  else:
      return np.pad(audio,(0,-start),mode='constant')[0:len(audio)]

In [ ]:
# ------------ Original audio ------------
# wavPath = "/content/PinkPanther30.wav"
# wavPath = "/content/piano2.wav"
# audio, sample_rate = librosa.load(wavPath)
audio = audio_sample

print("Original audio")
display(Audio(audio, rate=sample_rate))

# ------------ Noise Injection ------------
# audio = noise_injection(audio)

# ----------- Changing speed ------------
# audio = change_speed(audio)
# audio = change_speed(audio, speed_factor=0.5)
# audio = change_speed_and_fit(audio, stretch_factor=0.5)
audio = change_speed_and_fit(audio, stretch_factor=2)

# ------------ Pitch Shift --------------
# audio = shift_pitch(audio)

# ------------ Time Shift --------------
# audio = shift_time(audio)

print("Augmented audio")
librosa.display.waveshow(audio.astype('float'), sr=sample_rate)
display(Audio(audio, rate=sample_rate))